## BASIC SPARK IMPLEMENTATIONS

In [73]:
!pip install findspark

import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import expr

sc = SparkContext('local')
spark = SparkSession(sc)
sc

<SparkContext master=local appName=pyspark-shell>

In [74]:
df = (spark.read.format('csv')
      .option('header', True)
      .option('delimiter', '|')
      .option('inferSchema', True)
      .load('D:/flo100k.csv')
      )
df.limit(5).toPandas()

,master_id,order_channel,platform_type,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,online_product_group_amount_top_name_12,offline_product_group_name_12,last_order_date_new,store_type
0,b3ace094-a17f-11e9-a2fc-000d3a38a36f,Offline,Offline,Offline,2019-02-23 12:59:17,2019-02-23 12:59:17,NaT,2019-02-23 12:59:17,NaN,1.0,212.98,0.00,None,None,None,2019-02-23,A
1,c57d7c4c-a950-11e9-a2fc-000d3a38a36f,Offline,OmniChannel,Offline,2019-12-01 16:48:09,2019-12-01 16:48:09,NaT,2019-12-01 16:48:09,NaN,1.0,199.98,0.00,None,None,None,2019-12-01,A
2,602897a6-cdac-11ea-b31f-000d3a38a36f,Offline,Offline,Offline,2020-07-24 15:49:47,2020-07-24 15:49:47,NaT,2020-07-24 15:49:47,NaN,1.0,140.49,0.00,[ERKEK],None,ERKEK,2020-07-24,A
3,388e4c4e-af86-11e9-a2fc-000d3a38a36f,Mobile,Online,Mobile,2018-12-31 07:22:07,2018-12-31 07:22:07,2018-12-31 07:22:07,NaT,1.0,NaN,0.00,174.99,None,None,None,2018-12-31,A
4,80664354-adf0-11eb-8f64-000d3a299ebf,Desktop,Online,Desktop,2021-05-05 21:07:02,2021-05-05 22:39:36,2021-05-05 22:39:36,NaT,2.0,NaN,0.00,283.95,[],None,None,2021-05-05,A


In [75]:
df.show(2)

+--------------------+-------------+-------------+------------------+-------------------+-------------------+----------------------+-----------------------+---------------------------+----------------------------+---------------------------------+--------------------------------+---------------------------+---------------------------------------+-----------------------------+-------------------+----------+
|           master_id|order_channel|platform_type|last_order_channel|   first_order_date|    last_order_date|last_order_date_online|last_order_date_offline|order_num_total_ever_online|order_num_total_ever_offline|customer_value_total_ever_offline|customer_value_total_ever_online|interested_in_categories_12|online_product_group_amount_top_name_12|offline_product_group_name_12|last_order_date_new|store_type|
+--------------------+-------------+-------------+------------------+-------------------+-------------------+----------------------+-----------------------+------------------------

In [76]:
# FEATURE SELECTION
df.select ("store_type").limit(2).toPandas()

,store_type
0,A
1,A


In [77]:
df.printSchema()

root
 |-- master_id: string (nullable = true)
 |-- order_channel: string (nullable = true)
 |-- platform_type: string (nullable = true)
 |-- last_order_channel: string (nullable = true)
 |-- first_order_date: timestamp (nullable = true)
 |-- last_order_date: timestamp (nullable = true)
 |-- last_order_date_online: timestamp (nullable = true)
 |-- last_order_date_offline: timestamp (nullable = true)
 |-- order_num_total_ever_online: double (nullable = true)
 |-- order_num_total_ever_offline: double (nullable = true)
 |-- customer_value_total_ever_offline: double (nullable = true)
 |-- customer_value_total_ever_online: double (nullable = true)
 |-- interested_in_categories_12: string (nullable = true)
 |-- online_product_group_amount_top_name_12: string (nullable = true)
 |-- offline_product_group_name_12: string (nullable = true)
 |-- last_order_date_new: string (nullable = true)
 |-- store_type: string (nullable = true)



In [78]:
#Create a new column with SQL Expr using datediff
df= df.withColumn("Recency", expr("datediff('2021-6-1', last_order_date_new)"))

In [79]:
df.columns

['master_id',
 'order_channel',
 'platform_type',
 'last_order_channel',
 'first_order_date',
 'last_order_date',
 'last_order_date_online',
 'last_order_date_offline',
 'order_num_total_ever_online',
 'order_num_total_ever_offline',
 'customer_value_total_ever_offline',
 'customer_value_total_ever_online',
 'interested_in_categories_12',
 'online_product_group_amount_top_name_12',
 'offline_product_group_name_12',
 'last_order_date_new',
 'store_type',
 'Recency']

In [80]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [81]:
# CREATE NEW COLUMNS WITH SUM OF TWO COLUMNS
df.withColumn ("customer_value_total", df.customer_value_total_ever_offline + df.customer_value_total_ever_online ).select("customer_value_total").show(5)

+--------------------+
|customer_value_total|
+--------------------+
|              212.98|
|              199.98|
|              140.49|
|              174.99|
|              283.95|
+--------------------+
only showing top 5 rows



In [82]:
df = df.withColumn ("customer_value_total", df.customer_value_total_ever_offline + df.customer_value_total_ever_online )

In [83]:
# AMORTIZE ALL COLUMN NAMES

df = df.toDF(*[c.lower() for c in df.columns])

In [84]:
df.columns

['master_id',
 'order_channel',
 'platform_type',
 'last_order_channel',
 'first_order_date',
 'last_order_date',
 'last_order_date_online',
 'last_order_date_offline',
 'order_num_total_ever_online',
 'order_num_total_ever_offline',
 'customer_value_total_ever_offline',
 'customer_value_total_ever_online',
 'interested_in_categories_12',
 'online_product_group_amount_top_name_12',
 'offline_product_group_name_12',
 'last_order_date_new',
 'store_type',
 'recency',
 'customer_value_total']

In [85]:
# UNIQUE VALUE ANALYSIS
df.distinct().count()

100000

In [86]:
# IT SEEMS THERE IS NO DUPLICATED VALUE
df.count()

100000

In [87]:
df.select('master_id').distinct().count()

100000

In [88]:
df.limit(2).show()

+--------------------+-------------+-------------+------------------+-------------------+-------------------+----------------------+-----------------------+---------------------------+----------------------------+---------------------------------+--------------------------------+---------------------------+---------------------------------------+-----------------------------+-------------------+----------+-------+--------------------+
|           master_id|order_channel|platform_type|last_order_channel|   first_order_date|    last_order_date|last_order_date_online|last_order_date_offline|order_num_total_ever_online|order_num_total_ever_offline|customer_value_total_ever_offline|customer_value_total_ever_online|interested_in_categories_12|online_product_group_amount_top_name_12|offline_product_group_name_12|last_order_date_new|store_type|recency|customer_value_total|
+--------------------+-------------+-------------+------------------+-------------------+-------------------+-------------

In [89]:
# PICK AN OBSERVATION
df.filter(df.master_id =="b3ace094-a17f-11e9-a2fc-000d3a38a36f").toPandas()
# df.filter(df.master_id =="b3ace094-a17f-11e9-a2fc-000d3a38a36f" ).show()

,master_id,order_channel,platform_type,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,online_product_group_amount_top_name_12,offline_product_group_name_12,last_order_date_new,store_type,recency,customer_value_total
0,b3ace094-a17f-11e9-a2fc-000d3a38a36f,Offline,Offline,Offline,2019-02-23 12:59:17,2019-02-23 12:59:17,NaT,2019-02-23 12:59:17,NaN,1.0,212.98,0.0,None,None,None,2019-02-23,A,829,212.98


In [90]:
#Descriptive Statistics
df.describe().toPandas()

,summary,master_id,order_channel,platform_type,last_order_channel,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,online_product_group_amount_top_name_12,offline_product_group_name_12,last_order_date_new,store_type,recency,customer_value_total
0,count,100000,100000,100000,100000,29216,78297,100000,100000,43410,11705,22791,100000,100000,100000,100000
1,mean,None,None,None,None,2.66980421686747,1.6172778011928937,171.65589480013833,120.4383612000303,None,None,None,None,None,413.60178,292.0942559998722
2,stddev,None,None,None,None,3.4220875079703506,2.5701393859175456,332.48957911271873,359.37643994588313,None,None,None,None,None,255.23885293887244,463.1197490300459
3,min,00011a7c-5b3a-11ea-b125-000d3a38a36f,Android App,Offline,Android App,1.0,1.0,0.0,0.0,"[AKTIFCOCUK, AKTIFSPOR]",AKTIFCOCUK,AKTIFCOCUK,2018-05-31,A,2,0.0
4,max,ffffacf4-b23d-11ea-b736-000d3a38a36f,Offline,Online,Offline,142.0,377.0,51178.23,21962.85,[],KADIN,KADIN,2021-05-30,"B,C",1097,51178.23


In [91]:
# CHECK CROSS TABULATION WITH DESCRIBE METHOD
df.select ("recency","customer_value_total").describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
recency,100000,413.60178,255.23885293887244,2,1097
customer_value_total,100000,292.0942559998722,463.1197490300459,0.0,51178.23


In [92]:
#VALUE FILTERIN ON OBSERVATIONS
df.filter(df.recency > 100 ).count()

83917

In [93]:
# CHECK ALL THE CLASSES IN A VARIABLE
df.groupby("order_channel").count().show()

+-------------+-----+
|order_channel|count|
+-------------+-----+
|  Android App|11989|
|       Mobile| 8512|
|      Ios App| 3964|
|      Desktop| 4751|
|      Offline|70784|
+-------------+-----+



In [94]:
# CHECK ALL THE CLASSES COUNT IN A VARIABLE IN DESCENDING ORDER
df. groupby("order_channel").count().sort ("count", ascending = False).show (3)

+-------------+-----+
|order_channel|count|
+-------------+-----+
|      Offline|70784|
|  Android App|11989|
|       Mobile| 8512|
+-------------+-----+
only showing top 3 rows



In [95]:
# CHECK CROSS TABULATION WITH AGG FUNCTION
df.groupby("order_channel"). agg ({ "customer_value_total" : "mean"}). show ()

+-------------+-------------------------+
|order_channel|avg(customer_value_total)|
+-------------+-------------------------+
|  Android App|         532.846284093796|
|       Mobile|        391.7418761748324|
|      Ios App|        568.4640312815278|
|      Desktop|       376.91355504103205|
|      Offline|        218.1639484913738|
+-------------+-------------------------+



In [96]:
df.dtypes

[('master_id', 'string'),
 ('order_channel', 'string'),
 ('platform_type', 'string'),
 ('last_order_channel', 'string'),
 ('first_order_date', 'timestamp'),
 ('last_order_date', 'timestamp'),
 ('last_order_date_online', 'timestamp'),
 ('last_order_date_offline', 'timestamp'),
 ('order_num_total_ever_online', 'double'),
 ('order_num_total_ever_offline', 'double'),
 ('customer_value_total_ever_offline', 'double'),
 ('customer_value_total_ever_online', 'double'),
 ('interested_in_categories_12', 'string'),
 ('online_product_group_amount_top_name_12', 'string'),
 ('offline_product_group_name_12', 'string'),
 ('last_order_date_new', 'string'),
 ('store_type', 'string'),
 ('recency', 'int'),
 ('customer_value_total', 'double')]

### STRING INDEXING MULTIPLE COLUMNS

In [97]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer,  VectorAssembler, StandardScaler,OneHotEncoder
inputs = ['order_channel','platform_type']
outputs =["orderindexed","platformindexed" ]
indexer = StringIndexer(inputCols=inputs, outputCols= outputs)
indexed = indexer.fit(df).transform(df)

In [98]:
indexed.limit(5).toPandas()

,master_id,order_channel,platform_type,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,...,customer_value_total_ever_online,interested_in_categories_12,online_product_group_amount_top_name_12,offline_product_group_name_12,last_order_date_new,store_type,recency,customer_value_total,orderindexed,platformindexed
0,b3ace094-a17f-11e9-a2fc-000d3a38a36f,Offline,Offline,Offline,2019-02-23 12:59:17,2019-02-23 12:59:17,NaT,2019-02-23 12:59:17,NaN,1.0,...,0.00,None,None,None,2019-02-23,A,829,212.98,0.0,0.0
1,c57d7c4c-a950-11e9-a2fc-000d3a38a36f,Offline,OmniChannel,Offline,2019-12-01 16:48:09,2019-12-01 16:48:09,NaT,2019-12-01 16:48:09,NaN,1.0,...,0.00,None,None,None,2019-12-01,A,548,199.98,0.0,2.0
2,602897a6-cdac-11ea-b31f-000d3a38a36f,Offline,Offline,Offline,2020-07-24 15:49:47,2020-07-24 15:49:47,NaT,2020-07-24 15:49:47,NaN,1.0,...,0.00,[ERKEK],None,ERKEK,2020-07-24,A,312,140.49,0.0,0.0
3,388e4c4e-af86-11e9-a2fc-000d3a38a36f,Mobile,Online,Mobile,2018-12-31 07:22:07,2018-12-31 07:22:07,2018-12-31 07:22:07,NaT,1.0,NaN,...,174.99,None,None,None,2018-12-31,A,883,174.99,2.0,1.0
4,80664354-adf0-11eb-8f64-000d3a299ebf,Desktop,Online,Desktop,2021-05-05 21:07:02,2021-05-05 22:39:36,2021-05-05 22:39:36,NaT,2.0,NaN,...,283.95,[],None,None,2021-05-05,A,27,283.95,3.0,1.0


### ONE HOT ENCODING

In [99]:
encoder = OneHotEncoder() \
.setInputCols(["orderindexed","platformindexed" ]) \
.setOutputCols(["orderencoded","platformencoded" ])

encoder_model = encoder.fit(indexed)
encoded = encoder_model.transform(indexed)

In [100]:
encoded.limit(5).toPandas()

,master_id,order_channel,platform_type,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,...,online_product_group_amount_top_name_12,offline_product_group_name_12,last_order_date_new,store_type,recency,customer_value_total,orderindexed,platformindexed,orderencoded,platformencoded
0,b3ace094-a17f-11e9-a2fc-000d3a38a36f,Offline,Offline,Offline,2019-02-23 12:59:17,2019-02-23 12:59:17,NaT,2019-02-23 12:59:17,NaN,1.0,...,None,None,2019-02-23,A,829,212.98,0.0,0.0,"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0)"
1,c57d7c4c-a950-11e9-a2fc-000d3a38a36f,Offline,OmniChannel,Offline,2019-12-01 16:48:09,2019-12-01 16:48:09,NaT,2019-12-01 16:48:09,NaN,1.0,...,None,None,2019-12-01,A,548,199.98,0.0,2.0,"(1.0, 0.0, 0.0, 0.0)","(0.0, 0.0)"
2,602897a6-cdac-11ea-b31f-000d3a38a36f,Offline,Offline,Offline,2020-07-24 15:49:47,2020-07-24 15:49:47,NaT,2020-07-24 15:49:47,NaN,1.0,...,None,ERKEK,2020-07-24,A,312,140.49,0.0,0.0,"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0)"
3,388e4c4e-af86-11e9-a2fc-000d3a38a36f,Mobile,Online,Mobile,2018-12-31 07:22:07,2018-12-31 07:22:07,2018-12-31 07:22:07,NaT,1.0,NaN,...,None,None,2018-12-31,A,883,174.99,2.0,1.0,"(0.0, 0.0, 1.0, 0.0)","(0.0, 1.0)"
4,80664354-adf0-11eb-8f64-000d3a299ebf,Desktop,Online,Desktop,2021-05-05 21:07:02,2021-05-05 22:39:36,2021-05-05 22:39:36,NaT,2.0,NaN,...,None,None,2021-05-05,A,27,283.95,3.0,1.0,"(0.0, 0.0, 0.0, 1.0)","(0.0, 1.0)"


### VECTOR ASSEMBLER

In [101]:
# VECTOR ASSEMBLER FOR INDEPENDENT VARIABLES -FEATURES
# PICK VARIABLES AS ftures list

ftures = ["orderencoded","platformencoded" , 'customer_value_total_ever_offline',  'customer_value_total_ever_online',  'recency']

vectorAssembler = VectorAssembler(inputCols= ftures, outputCol = "features")
vectorized =vectorAssembler.transform(encoded)

In [102]:
vectorized.limit(5).toPandas()

,master_id,order_channel,platform_type,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,...,offline_product_group_name_12,last_order_date_new,store_type,recency,customer_value_total,orderindexed,platformindexed,orderencoded,platformencoded,features
0,b3ace094-a17f-11e9-a2fc-000d3a38a36f,Offline,Offline,Offline,2019-02-23 12:59:17,2019-02-23 12:59:17,NaT,2019-02-23 12:59:17,NaN,1.0,...,None,2019-02-23,A,829,212.98,0.0,0.0,"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 212.98, 0.0, 82..."
1,c57d7c4c-a950-11e9-a2fc-000d3a38a36f,Offline,OmniChannel,Offline,2019-12-01 16:48:09,2019-12-01 16:48:09,NaT,2019-12-01 16:48:09,NaN,1.0,...,None,2019-12-01,A,548,199.98,0.0,2.0,"(1.0, 0.0, 0.0, 0.0)","(0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 199.98, 0.0, 54..."
2,602897a6-cdac-11ea-b31f-000d3a38a36f,Offline,Offline,Offline,2020-07-24 15:49:47,2020-07-24 15:49:47,NaT,2020-07-24 15:49:47,NaN,1.0,...,ERKEK,2020-07-24,A,312,140.49,0.0,0.0,"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 140.49, 0.0, 31..."
3,388e4c4e-af86-11e9-a2fc-000d3a38a36f,Mobile,Online,Mobile,2018-12-31 07:22:07,2018-12-31 07:22:07,2018-12-31 07:22:07,NaT,1.0,NaN,...,None,2018-12-31,A,883,174.99,2.0,1.0,"(0.0, 0.0, 1.0, 0.0)","(0.0, 1.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 174.99, 88..."
4,80664354-adf0-11eb-8f64-000d3a299ebf,Desktop,Online,Desktop,2021-05-05 21:07:02,2021-05-05 22:39:36,2021-05-05 22:39:36,NaT,2.0,NaN,...,None,2021-05-05,A,27,283.95,3.0,1.0,"(0.0, 0.0, 0.0, 1.0)","(0.0, 1.0)","(0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 283.95, 27.0)"


### CASTING A VARIABLE TO INTEGER


In [104]:
# Variable called : customer_value_total has been casted as A NEW INTEGER VARIABLE CALLED "label"
vectorized= vectorized.withColumn("label", vectorized["customer_value_total"].cast("integer") )

In [105]:
vectorized.dtypes

[('master_id', 'string'),
 ('order_channel', 'string'),
 ('platform_type', 'string'),
 ('last_order_channel', 'string'),
 ('first_order_date', 'timestamp'),
 ('last_order_date', 'timestamp'),
 ('last_order_date_online', 'timestamp'),
 ('last_order_date_offline', 'timestamp'),
 ('order_num_total_ever_online', 'double'),
 ('order_num_total_ever_offline', 'double'),
 ('customer_value_total_ever_offline', 'double'),
 ('customer_value_total_ever_online', 'double'),
 ('interested_in_categories_12', 'string'),
 ('online_product_group_amount_top_name_12', 'string'),
 ('offline_product_group_name_12', 'string'),
 ('last_order_date_new', 'string'),
 ('store_type', 'string'),
 ('recency', 'int'),
 ('customer_value_total', 'double'),
 ('orderindexed', 'double'),
 ('platformindexed', 'double'),
 ('orderencoded', 'vector'),
 ('platformencoded', 'vector'),
 ('features', 'vector'),
 ('label', 'int')]

In [106]:
# PICK FINAL DF WITH FEAUTURES AND LABEL
final_df= vectorized.select ("features","label")
final_df.limit (5).toPandas()

,features,label
0,"(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 212.98, 0.0, 82...",212
1,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 199.98, 0.0, 54...",199
2,"(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 140.49, 0.0, 31...",140
3,"(0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 174.99, 88...",174
4,"(0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 283.95, 27.0)",283


In [107]:
spark.stop()